### Implement Agglomerative hierarchical clustering algorithm using appropriate dataset. 

In [2]:
import networkx as nx

# Custom PageRank function
def pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, nstart=None, weight='weight', dangling=None):
    if len(G) == 0: 
        return {} 
  
    # Convert to directed graph if G is undirected
    if not G.is_directed(): 
        D = G.to_directed() 
    else: 
        D = G
        
    # Create a copy in (right) stochastic form 
    W = nx.stochastic_graph(D, weight=weight) 
    N = W.number_of_nodes()
    
    # Choose fixed starting vector if not given 
    if nstart is None: 
        x = dict.fromkeys(W, 1.0 / N) 
    else: 
        # Normalized nstart vector 
        s = float(sum(nstart.values())) 
        x = dict((k, v / s) for k, v in nstart.items()) 
        
    # Personalization vector setup
    if personalization is None:
        # Assign uniform personalization vector if not given 
        p = dict.fromkeys(W, 1.0 / N)
    else: 
        missing = set(G) - set(personalization) 
        if missing: 
            raise nx.NetworkXError('Personalization dictionary must have a value for every node. Missing nodes %s' % missing) 
        s = float(sum(personalization.values())) 
        p = dict((k, v / s) for k, v in personalization.items())
        
    # Dangling nodes handling
    if dangling is None:
        # Use personalization vector if dangling vector not specified 
        dangling_weights = p 
    else: 
        missing = set(G) - set(dangling) 
        if missing: 
            raise nx.NetworkXError('Dangling node dictionary must have a value for every node. Missing nodes %s' % missing) 
        s = float(sum(dangling.values())) 
        dangling_weights = dict((k, v/s) for k, v in dangling.items())
        
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
    
    # Power iteration: make up to max_iter iterations 
    for _ in range(max_iter): 
        xlast = x 
        x = dict.fromkeys(xlast.keys(), 0) 
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes) 
        for n in x:
            # Matrix multiply with left multiply x^T=xlast^T*W 
            for nbr in W[n]: 
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight] 
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n] 
  
        # Check convergence, L1 norm 
        err = sum([abs(x[n] - xlast[n]) for n in x]) 
        if err < N * tol: 
            return x 
    raise nx.NetworkXError('Pagerank: power iteration failed to converge in %d iterations.' % max_iter)

# Generate a test graph
G = nx.barabasi_albert_graph(60, 41)  # Creates a Barabási–Albert graph with 60 nodes

# Calculate PageRank using the custom function
pr = pagerank(G, alpha=0.4)  # Using alpha=0.4 as specified in your example

# Print the resulting PageRank values
print("PageRank values:")
print(pr)


PageRank values:
{0: 0.028225221344340273, 1: 0.01337258610187707, 2: 0.01278019140055241, 3: 0.013175697960476889, 4: 0.01298132107058749, 5: 0.01318781361064749, 6: 0.012377905435756842, 7: 0.0127717119774061, 8: 0.013157283612231835, 9: 0.013178031011730873, 10: 0.012979380531714673, 11: 0.013372371204340127, 12: 0.012767909599195675, 13: 0.012762964863236108, 14: 0.012982995063759283, 15: 0.01296423613610792, 16: 0.013364899784992888, 17: 0.012982148011506392, 18: 0.013370567578156552, 19: 0.013374270901695701, 20: 0.013574929613221515, 21: 0.012353148549986865, 22: 0.012968749481791919, 23: 0.012767968029910226, 24: 0.012792134401655936, 25: 0.011181235754962961, 26: 0.011974401516145568, 27: 0.01317088017684138, 28: 0.013170047860868445, 29: 0.01357363994514994, 30: 0.013379513103028642, 31: 0.012964022020484682, 32: 0.012362002508011205, 33: 0.01297217679447552, 34: 0.01255903225224812, 35: 0.012574353188341741, 36: 0.013584516803513346, 37: 0.013574929613221515, 38: 0.012369706